In [1]:
# GAIP NUS project - GROUP 2 (BATCH 2)

import pandas as pd
import re
import string
import nltk
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [3]:
import numpy

In [4]:
import xgboost

In [5]:
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('Book1.csv', encoding = 'ISO-8859-1')

In [11]:
df

,TITLE,NAME,COMMENT,LABEL,CONFIDENCE
0,Difficulty with Behringer Guitar 2 usb cable,jtc831,im an experienced guitarist but still very new...,Classification,Confidence
2,DM12 Windows editor connection issues,Glestik55,does anyone know if the new firmware addresses...,awareness,0.592
4,Synths Resurrected Vol. 3,UliBehringer,the prophet we recently acquired form the fam...,awareness,0.417
5,Lack of suport by Music Tribe,fastedtex,g sounding very badreplaced both tubesstill so...,negative,0.672
6,X Air Edit on Win10 Tablet,Karma180,hi is anyone using x air edit on a windows ta...,consideration,0.662
7,Play acoustic reverb ?. (pre delay) on guitar ...,faceache,hi there i recently acquired a play acoustic a...,awareness,0.81
8,Neutron and Volca Sample jam,NelsPeterson,httpsyoutuberdoxlika,awareness,0.707
10,Feedback noise with Voicelive Play GTX,ry_cooder,hello ive bought a secondhand vlp gtx great un...,decision,0.309
12,VoicePlay Live Practice Mode percentages,Tlink211,with voiceplay live i reset my practice mode a...,awareness,0.794
13,TC Helicon Play Acoustic Issue (Pitch Correction),ahuaman96,ive been using my own presets with correct nat...,decision,0.816


In [8]:
df = df.dropna()

In [10]:
# changing the wrong labels to the right one.
df['LABEL'].unique()
df[df['LABEL'] == 'awarness'] = 'awareness'
df[df['LABEL'] == 'decsion'] = 'decision'
df[df['LABEL'] == 'complaint'] = 'negative'
df[df['LABEL'] == 'decision:awareness'] = 'decision'
df[df['LABEL'] == 'consideration:awareness'] = 'consideration'
df[df['LABEL'] == 'awareness:consideration'] = 'awareness'
df[df['LABEL'] == 'awareness:decision'] = 'awareness'
df[df['LABEL'] == 'awareness:negative'] = 'awareness'
df[df['LABEL'] == 'decision:consideration'] = 'decision'
df[df['LABEL'] == 'negative:awareness'] = 'negative'
df[df['LABEL'] == 'decision:negative'] = 'decision'

# Converting all the text to lowercase in the COMMENT.
df['COMMENT'] = df['COMMENT'].str.lower()
# df['COMMENT']

# Removing all rhe numerical values from the COMMENT.
df['COMMENT'] = df['COMMENT'].str.replace('\d+','')
# df['COMMENT']

# Removing all the punctuations from the COMMENT.
df['COMMENT'] = df['COMMENT'].str.replace('[{}]'.format(string.punctuation), '')
# df['COMMENT']

# REmoving all the whitespaces in the COMMENT.
df['COMMENT'] = df['COMMENT'].str.strip()
# df[df['COMMENT']=='decision']

df = df[df.TITLE != 'decision']


In [12]:
df = df[df['LABEL'] != 'Classification']

In [13]:
df['LABEL'].unique()

array(['awareness', 'negative', 'consideration', 'decision'], dtype=object)

In [14]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
#     classifier.fit(feature_vector_train, label)
    hist = classifier.fit(feature_vector_train, label, epochs=10)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [15]:
# Splitting the dataset into training and validation model.
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['COMMENT'], df['LABEL'])

In [17]:
df['COMMENT']=[" ".join(review) for review in df['COMMENT'].values]

C:\Users\Devish6\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
# Encoding the label variable. From Categorical to numerical Data.
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [19]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['COMMENT'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [20]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, stop_words = 'english')
tfidf_vect.fit(df['COMMENT'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [21]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000, stop_words = 'english')
tfidf_vect_ngram.fit(df['COMMENT'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [22]:
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000, stop_words = 'english')
tfidf_vect_ngram_chars.fit(df['COMMENT'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [23]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('wiki-news-300d-1M.vec',encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(df['COMMENT'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [24]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    hidden_layer1 = layers.Dense(100, activation="relu")(hidden_layer)
    
    hidden_layer2 = layers.Dense(100, activation="relu")(hidden_layer1)
    
    # create output layer
    output_layer = layers.Dense(1, activation="relu")(hidden_layer2)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

In [26]:
# Shallow Neural Nets
classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

Epoch 1/10
360/360 [==============================] - 0s 1ms/step - loss: 6.4920
Epoch 2/10
360/360 [==============================] - 0s 357us/step - loss: 6.4920
Epoch 3/10
360/360 [==============================] - 0s 355us/step - loss: 6.4920
Epoch 4/10
360/360 [==============================] - 0s 377us/step - loss: 6.4920
Epoch 5/10
360/360 [==============================] - 0s 388us/step - loss: 6.4920
Epoch 6/10
360/360 [==============================] - 0s 388us/step - loss: 6.4920
Epoch 7/10
360/360 [==============================] - 0s 346us/step - loss: 6.4920
Epoch 8/10
360/360 [==============================] - 0s 341us/step - loss: 6.4920
Epoch 9/10
360/360 [==============================] - 0s 343us/step - loss: 6.4920
Epoch 10/10
360/360 [==============================] - 0s 454us/step - loss: 6.4920
NN, Ngram Level TF IDF Vectors 0.7583333333333333


In [27]:
# CNN
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)

W0625 01:16:03.260932 27596 deprecation.py:506] From C:\Users\Devish6\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch 1/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6972
Epoch 2/10
360/360 [==============================] - 0s 446us/step - loss: 0.6785
Epoch 3/10
360/360 [==============================] - 0s 443us/step - loss: 0.6606
Epoch 4/10
360/360 [==============================] - 0s 432us/step - loss: 0.6499
Epoch 5/10
360/360 [==============================] - 0s 438us/step - loss: 0.6434
Epoch 6/10
360/360 [==============================] - 0s 438us/step - loss: 0.6540
Epoch 7/10
360/360 [==============================] - 0s 438us/step - loss: 0.6201
Epoch 8/10
360/360 [==============================] - 0s 435us/step - loss: 0.5924
Epoch 9/10
360/360 [==============================] - 0s 443us/step - loss: 0.5950
Epoch 10/10
360/360 [==============================] - 0s 452us/step - loss: 0.6219
CNN, Word Embeddings 0.7583333333333333


In [28]:
# LTSM
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-LSTM, Word Embeddings",  accuracy)

Epoch 1/10
360/360 [==============================] - 1s 4ms/step - loss: 0.6966
Epoch 2/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6866
Epoch 3/10
360/360 [==============================] - 1s 1ms/step - loss: 0.6832
Epoch 4/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6680
Epoch 5/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6610
Epoch 6/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6562
Epoch 7/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6625
Epoch 8/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6689
Epoch 9/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6633
Epoch 10/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6517
RNN-LSTM, Word Embeddings 0.7583333333333333


In [29]:
# Reccurent nural network
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-GRU, Word Embeddings",  accuracy)

Epoch 1/10
360/360 [==============================] - 1s 4ms/step - loss: 0.6904
Epoch 2/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6918
Epoch 3/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6729
Epoch 4/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6728
Epoch 5/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6712
Epoch 6/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6577
Epoch 7/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6452
Epoch 8/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6439
Epoch 9/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6305
Epoch 10/10
360/360 [==============================] - 0s 1ms/step - loss: 0.6052
RNN-GRU, Word Embeddings 0.7583333333333333


In [30]:
# Bi-directional RNN
def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-Bidirectional, Word Embeddings",  accuracy)

Epoch 1/10
360/360 [==============================] - 2s 6ms/step - loss: 0.6914
Epoch 2/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6809
Epoch 3/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6726
Epoch 4/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6735
Epoch 5/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6674
Epoch 6/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6567
Epoch 7/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6678
Epoch 8/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6473
Epoch 9/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6474A: 0s - loss:
Epoch 10/10
360/360 [==============================] - 1s 2ms/step - loss: 0.6517
RNN-Bidirectional, Word Embeddings 0.7583333333333333


In [31]:
def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)

Epoch 1/10
360/360 [==============================] - 1s 3ms/step - loss: 0.7247
Epoch 2/10
360/360 [==============================] - 0s 449us/step - loss: 0.6793
Epoch 3/10
360/360 [==============================] - 0s 446us/step - loss: 0.6760
Epoch 4/10
360/360 [==============================] - 0s 440us/step - loss: 0.6525
Epoch 5/10
360/360 [==============================] - 0s 449us/step - loss: 0.6380
Epoch 6/10
360/360 [==============================] - 0s 454us/step - loss: 0.6317
Epoch 7/10
360/360 [==============================] - 0s 463us/step - loss: 0.6011
Epoch 8/10
360/360 [==============================] - 0s 573us/step - loss: 0.6063
Epoch 9/10
360/360 [==============================] - 0s 449us/step - loss: 0.5881
Epoch 10/10
360/360 [==============================] - 0s 463us/step - loss: 0.5850
CNN, Word Embeddings 0.7583333333333333


In [32]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

TypeError: fit() got an unexpected keyword argument 'epochs'

In [38]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.09803921568627451
LR, WordLevel TF-IDF:  0.09803921568627451
LR, N-Gram Vectors:  0.09803921568627451
LR, CharLevel Vectors:  0.09803921568627451


C:\Users\Devish6\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Devish6\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [39]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.09803921568627451


C:\Users\Devish6\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [40]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

RF, Count Vectors:  0.09803921568627451
RF, WordLevel TF-IDF:  0.0784313725490196


C:\Users\Devish6\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Devish6\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [41]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)

Xgb, Count Vectors:  0.09803921568627451
Xgb, WordLevel TF-IDF:  0.0784313725490196
Xgb, CharLevel Vectors:  0.09803921568627451
